In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# For google drive - location of data
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Sea/data')

Mounted at /content/drive


In [3]:
#Loading Data

X  = pd.read_csv('training_input_ssta.csv',header=None)
Y  = pd.read_csv('training_output.csv',header=None)
Z  = pd.read_csv('test_input_ssta_phase_two.csv',header=None)
All = pd.read_csv('ssta_1940_2023.csv',header=None)


C  = pd.read_csv('coords.csv',header=None)
C  = C.T.reset_index()

C.columns = range(3)

In [39]:
#Build features  main (location under study) + min,max and mean of 8 closest
def minmax(df):
  out=pd.DataFrame()
  for i in range(3):
    rows= np.array([0,1,2,3,5,6,7,8])+i*9
    out['main_'+str(i)] = df[i*9+4]
    out['mean_'+str(i)] = df[rows].mean(axis=1)
    out['max_'+str(i)]  = df[rows].max(axis=1)
    out['min_'+str(i)]  = df[rows].min(axis=1)
  return out

In [5]:
#Mapping values to the new structure
def MapValues(X,LRUD):
  shape=X.shape
  B=np.zeros((shape[0],shape[1],9))+np.nan
  for i in range(len(X)):
    A=LRUD*0
    for c in LRUD.columns:
      A[c] = LRUD[c].map(X.loc[i].T)
    B[i]=A.values
  return B

In [6]:
# global map dataframe
df = pd.DataFrame(columns=C[1].unique(),index=C[2].unique())
for  c in df.columns:
    df[c]=C.loc[C[1]==c].set_index(2)[0].astype(int)

In [7]:
# New DataFrame:  location under study + 8 neighboring locations per row
LRUD=pd.DataFrame(index=df.values.flatten())
for i in range(-1,2):
  for j in range(-1,2):
     LRUD['C'+str(i)+'_'+str(j)]=np.nan

     LRUD['C'+str(i)+'_'+str(j)] = df.shift(-i,axis=1).shift(-j,axis=0).values.flatten()
LRUD=LRUD.loc[X.columns]


In [73]:
#Apply the mappings  (keeping up to last semptember)
Ball = MapValues(All,LRUD)[9:-3][::12]

In [81]:
Ball.shape

(83, 5774, 9)

In [84]:
#  concatenating 12 months per row - All
Xall= pd.DataFrame()
for i in range(0,3):
      for j in range(9):
        Xall[i-3 +j*3]=Ball[i:i-3,:,j].flatten()

Yall = All[9:-3][::12][3:].values.flatten()
Xall.columns=list(range(3*9))

In [85]:
Xout= pd.DataFrame()
for i in range(0,2):
      for j in range(9):
        Xout[i-3 +j*3]=Ball[i:i-2,:,j].flatten()

i = 2
for j in range(9):
  Xout[i-3 +j*3]=Ball[2:,:,j].flatten()

Xout.columns=list(range(3*9))

In [86]:
#  final input stracture
Xout=minmax(Xout)
Xall=minmax(Xall)

In [93]:
# size per year
year=int(len(Xall)/9)

In [105]:
# Training Bayesian Ridge Models
from sklearn import linear_model
reg3 = linear_model.BayesianRidge()
reg9 = linear_model.BayesianRidge()

reg3.fit(Xall.fillna(0)[-year*10:].values, Yall[-year*10:])
reg9.fit(Xall.fillna(0)[-year*15:].values, Yall[-year*15:])

BayesianRidge()

In [110]:
predictions =   0.5*(reg3.predict(Xout.fillna(0).values) + reg9.predict(Xout.fillna(0).values)).reshape((-1,5774))

In [134]:
Table = pd.DataFrame()
Table['predictions']= pd.DataFrame(predictions)[[5640,5690,5773]].iloc[-1]
Table['lat'] = [56,58,62]
Table['lon']= [19.6875,19.6875,19.6875]
Table.to_latex()

'\\begin{tabular}{lrrr}\n\\toprule\n & predictions & lat & lon \\\\\n\\midrule\n5640 & 0.592666 & 56 & 19.687500 \\\\\n5690 & 0.648081 & 58 & 19.687500 \\\\\n5773 & 0.425684 & 62 & 19.687500 \\\\\n\\bottomrule\n\\end{tabular}\n'

Post Competition - Full Data